<a href="https://colab.research.google.com/github/anuytok/ChIP-Seq/blob/main/Chip_seq_%D0%95%D1%84%D0%B8%D0%BC%D0%BE%D0%B2%D0%B0_%D0%90%D0%BD%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Импортируем гугл диск. Оттуда по пути вытащим папку с необходимыми для анализа файлы (UxuR_1, UxuR_2 и контроль UxuR_control).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Скачаем trimmomatic и положим в рабочую директорию

In [ ]:
!wget https://github.com/usadellab/Trimmomatic/files/5854859/Trimmomatic-0.39.zip

In [ ]:
!unzip /content/Trimmomatic-0.39.zip

Создадим папку, куда положим результаты trimmomatic для ридов 50SR UxuR

In [ ]:
!mkdir trimmed

Экстрагируем риды и обрежем их с помощью Trimmomatic c параметрами 

ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:true LEADING:3 TRAILING:3 MINLEN:36 


In [ ]:
!java -jar /content/Trimmomatic-0.39/trimmomatic-0.39.jar SE /content/drive/MyDrive/HSE_2022/UxuR_1.fastq /content/trimmed/UxuR_1.fastq ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:true LEADING:3 TRAILING:3 MINLEN:36 

In [ ]:
!java -jar /content/Trimmomatic-0.39/trimmomatic-0.39.jar SE /content/drive/MyDrive/HSE_2022/UxuR_2.fastq /content/trimmed/UxuR_2.fastq ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:true LEADING:3 TRAILING:3 MINLEN:36 

In [ ]:
!java -jar /content/Trimmomatic-0.39/trimmomatic-0.39.jar SE /content/drive/MyDrive/HSE_2022/UxuR_control.fastq /content/trimmed/UxuR_control.fastq ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:true LEADING:3 TRAILING:3 MINLEN:36 

Наложим на геном E. coli K-12 MG1655 genome с помощью Bowtie


In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
!conda install -y -c bioconda bowtie2

Загрузим геном Ecoli_K12_MG1655 (с компьютера)

Проиндексируем геном

In [ ]:
!bowtie2-build Ecoli_K12_MG1655.fasta Ecoli

In [ ]:
!conda install -y -c bioconda samtools 

In [ ]:
%cd ../content/

Прокартируем риды с помощью bowtie2, а затем с помощью samtools получим bam файлы, которые отсортируем

In [ ]:
!bowtie2 -x Ecoli -U /content/trimmed/UxuR_1.fastq -S /content/trimmed/UxuR_1.sam
!samtools view -h -S -b -o /content/trimmed/UxuR_1.bam /content/trimmed/UxuR_1.sam
!samtools sort /content/trimmed/UxuR_1.bam -o /content/trimmed/UxuR_1.bam

In [ ]:
!bowtie2 -x Ecoli -U /content/trimmed/UxuR_2.fastq -S /content/trimmed/UxuR_2.sam
!samtools view -h -S -b -o /content/trimmed/UxuR_2.bam /content/trimmed/UxuR_2.sam
!samtools sort /content/trimmed/UxuR_2.bam -o /content/trimmed/UxuR_2.bam

In [ ]:
!bowtie2 -x Ecoli -U /content/trimmed/UxuR_control.fastq -S /content/trimmed/UxuR_control.sam
!samtools view -h -S -b -o /content/trimmed/UxuR_control.bam /content/trimmed/UxuR_control.sam
!samtools sort /content/trimmed/UxuR_control.bam -o /content/trimmed/UxuR_control.bam

Найдем пики с помощью MACS2 и сделаем таблицу с генами, к которым они относятся

In [ ]:
!conda install -c bioconda macs2

In [ ]:
!mkdir macs2

In [ ]:
%cd /content/trimmed

In [ ]:
!macs2 callpeak -t UxuR_1.bam -c UxuR_control.bam -f BAM --nomodel -p 0.05 --extsize 147 --outdir /content/macs2

Переименуем файлы в соответствии с названием 1 образца и запустим поиск пиков для 2 образца

In [ ]:
!mv /content/macs2/NA_peaks.narrowPeak  /content/macs2/UxuR_1.narrowPeak 
!mv /content/macs2/NA_summits.bed  /content/macs2/UxuR_1.bed
!mv /content/macs2/NA_peaks.xls  /content/macs2/UxuR_1.xls

In [ ]:
!macs2 callpeak -t UxuR_2.bam -c UxuR_control.bam -f BAM --nomodel -p 0.05 --extsize 147 --outdir /content/macs2

Переименуем файлы в соответствии с названием 2 образца

In [ ]:
!mv /content/macs2/NA_peaks.narrowPeak  /content/macs2/UxuR_2.narrowPeak 
!mv /content/macs2/NA_summits.bed  /content/macs2/UxuR_2.bed
!mv /content/macs2/NA_peaks.xls  /content/macs2/UxuR_2.xls

Посмотрим на число пиков в 1 и 2 образце

In [ ]:
! wc -l /content/macs2/UxuR_1.narrowPeak

22 /content/macs2/UxuR_1.narrowPeak


In [ ]:
! wc -l /content/macs2/UxuR_2.narrowPeak

38 /content/macs2/UxuR_2.narrowPeak


Теперь необходимо пересечь координаты полученных пиков по двум образцам. Сделаем это с помощью bedtools.

In [ ]:
!conda install -y -c bioconda bedtools

In [ ]:
%cd /content/macs2

In [ ]:
! bedtools intersect -a UxuR_1.narrowPeak -b UxuR_2.narrowPeak > UxuR.bed

Вытащим из аннотации по данным координатам гены. Учтем, что гены могут перекрываться (то есть координаты пиков могут попадать в промежутки разных генов из аннотации)

In [ ]:
%cd /content/

Сделаем из файла датафрейм

In [ ]:
!cat /content/macs2/UxuR.bed > UxuR.txt

Скачаем аннотацию генома E. Coli

In [ ]:
!wget ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.gff.gz

In [ ]:
!gunzip GCF_000005845.2_ASM584v2_genomic.gff.gz

In [ ]:
!cat GCF_000005845.2_ASM584v2_genomic.gff > annotation_pre.txt
!awk 'NR>7' annotation_pre.txt  > annotation.txt

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
annotation = pd.read_csv('annotation.txt', sep='\t')
uxur = pd.read_csv('UxuR.txt', sep='\t', names=["0", "Start", "End", "3", '4', '5', '6', 'p-value (-log10)', '8', '9'])
uxur = uxur[['Start', 'End', 'p-value (-log10)']]

uxur.head()

,Start,End,p-value (-log10)
0,313256,313585,1.58234
1,1215793,1215944,3.12916
2,1633815,1634134,3.47978
3,1636787,1636935,1.52251
4,1696006,1696162,1.60706


In [ ]:
annotation = annotation.loc[annotation['region'] =='gene']
annotation = annotation.rename({'1': 'start', '4641652': 'end'}, axis=1)

Создадим в аннотации колонку с названиями генов

In [ ]:
annotation = annotation.rename({'ID=NC_000913.3:1..4641652;Dbxref=taxon:511145;Is_circular=true;Name=ANONYMOUS;gbkey=Src;genome=chromosome;mol_type=genomic DNA;strain=K-12;substrain=MG1655' : 'ID'}, axis=1)
annotation['Genes'] = annotation['ID'].apply(lambda s: s.split(";")[2])
annotation['Gene'] = annotation['Genes'].apply(lambda s: s.split("=")[1])
del annotation['Genes']
annotation.head()

,NC_000913.3,RefSeq,region,start,end,.,+,..1,ID,Gene
0,NC_000913.3,RefSeq,gene,190.0,255.0,.,+,.,"ID=gene-b0001;Dbxref=ASAP:ABE-0000006,ECOCYC:E...",thrL
2,NC_000913.3,RefSeq,gene,337.0,2799.0,.,+,.,"ID=gene-b0002;Dbxref=ASAP:ABE-0000008,ECOCYC:E...",thrA
4,NC_000913.3,RefSeq,gene,2801.0,3733.0,.,+,.,"ID=gene-b0003;Dbxref=ASAP:ABE-0000010,ECOCYC:E...",thrB
6,NC_000913.3,RefSeq,gene,3734.0,5020.0,.,+,.,"ID=gene-b0004;Dbxref=ASAP:ABE-0000012,ECOCYC:E...",thrC
8,NC_000913.3,RefSeq,gene,5234.0,5530.0,.,+,.,"ID=gene-b0005;Dbxref=ASAP:ABE-0000015,ECOCYC:G...",yaaX


Найдем гены, для которых координаты пиков попадают четко в их интервал по аннотации

In [ ]:
genes_cross = []
for j in uxur.index:
  for i in annotation.index:
      if (uxur['Start'][j] > annotation['start'][i]) and (uxur['End'][j] < annotation['end'][i]):
        genes_cross.append(annotation['start'][i])

genes_cross_df = annotation[annotation.loc[:,"start"].isin(genes_cross)]
genes_cross_df

,NC_000913.3,RefSeq,region,start,end,.,+,..1,ID,Gene
2491,NC_000913.3,RefSeq,gene,1215789.0,1216025.0,.,+,.,"ID=gene-b1164;Dbxref=ASAP:ABE-0003907,ECOCYC:G...",ycgZ
3356,NC_000913.3,RefSeq,gene,1633622.0,1634212.0,.,+,.,"ID=gene-b1545;Dbxref=ASAP:ABE-0005164,ECOCYC:G...",pinQ
3364,NC_000913.3,RefSeq,gene,1636756.0,1636989.0,.,+,.,"ID=gene-b4533;Dbxref=ASAP:ABE-0285054,ECOCYC:G...",ynfO
3525,NC_000913.3,RefSeq,gene,1699355.0,1700947.0,.,+,.,"ID=gene-b1621;Dbxref=ASAP:ABE-0005429,ECOCYC:E...",malX
3685,NC_000913.3,RefSeq,gene,1775587.0,1777182.0,.,+,.,"ID=gene-b1694;Dbxref=ASAP:ABE-0005656,ECOCYC:E...",ydiF
4420,NC_000913.3,RefSeq,gene,2104494.0,2105084.0,.,-,.,"ID=gene-b2033;Dbxref=ASAP:ABE-0006749,ECOCYC:E...",wbbJ
4426,NC_000913.3,RefSeq,gene,2107226.0,2108329.0,.,-,.,"ID=gene-b2036;Dbxref=ASAP:ABE-0006755,ECOCYC:E...",glf
6153,NC_000913.3,RefSeq,gene,2992094.0,2993470.0,.,+,.,"ID=gene-b2852;Dbxref=ASAP:ABE-0009368,ECOCYC:G...",ygeH
7799,NC_000913.3,RefSeq,gene,3800267.0,3800965.0,.,-,.,"ID=gene-b3625;Dbxref=ASAP:ABE-0011856,ECOCYC:E...",waaY


Найдем гены, для которых начальные координаты пиков могут перекрываться с несколькими генами

In [ ]:
start_cross = []
for j in uxur.index:
  for i in annotation.index:
      if (uxur['Start'][j] > annotation['start'][i] and uxur['Start'][j] < annotation['end'][i]):
        start_cross.append(annotation['start'][i])

Удалим совпадающие с 1 массивом элементы

In [ ]:
for a in genes_cross:
  for b in start_cross:
    if a == b:
      start_cross.remove(b)

In [ ]:
start_cross_df = annotation[annotation.loc[:,"start"].isin(start_cross)]
start_cross_df

,NC_000913.3,RefSeq,region,start,end,.,+,..1,ID,Gene
3517,NC_000913.3,RefSeq,gene,1694260.0,1696071.0,.,-,.,"ID=gene-b1617;Dbxref=ASAP:ABE-0005410,ECOCYC:E...",uidA
6148,NC_000913.3,RefSeq,gene,2989935.0,2990360.0,.,-,.,"ID=gene-b2849;Dbxref=ASAP:ABE-0009356,ECOCYC:G...",yqeK


Найдем гены, для которых конечные координаты пиков могут перекрываться с несколькими генами

In [ ]:
end_cross = []
for j in uxur.index:
  for i in annotation.index:
      if (uxur['End'][j] > annotation['start'][i] and uxur['End'][j] < annotation['end'][i]):
        end_cross.append(annotation['start'][i])

Удалим совпадающие с 1 массивом элементы

In [ ]:
for a in genes_cross:
  for b in end_cross:
    if a == b:
      end_cross.remove(b)

In [ ]:
end_cross_df = annotation[annotation.loc[:,"start"].isin(end_cross)]
end_cross_df

,NC_000913.3,RefSeq,region,start,end,.,+,..1,ID,Gene
3521,NC_000913.3,RefSeq,gene,1697273.0,1698040.0,.,-,.,"ID=gene-b1619;Dbxref=ASAP:ABE-0005419,ECOCYC:E...",hdhA


Сделаем итоговую таблицу, где в столбцах start и end хранится информация о начальных и конечных координатах полученных пиков, также p-value, и гены. В строках лежат координаты пиков, которые четко попали внутрь промежутка координат гена по аннотации, также координаты пиков, начальная координата которых попала в промежуток координат гена по аннотации, а конечная координата нет, и еще координаты пиков, конечная координата которых попала в промежуток координат гена по аннотации, а начальная координата нет.

In [ ]:
cross_df = pd.concat([genes_cross_df, start_cross_df, end_cross_df], axis = 0)

uxur_genes = uxur.drop(uxur.index[[0,11,13]])
uxur_genes = uxur_genes.reset_index()
del uxur_genes['index']

df2 = pd.DataFrame(['ycgZ', 'pinQ', 'ynfO', 'uidA', 'hdhA', 'malX', 'ydiF', 'wbbJ', 'glf', 'yqeK', 'ygeH', 'waaY'], columns=['Genes'])

uxur_genes_df = (pd.concat([uxur_genes, df2], axis=1))
uxur_genes_df

,Start,End,p-value (-log10),Genes
0,1215793,1215944,3.12916,ycgZ
1,1633815,1634134,3.47978,pinQ
2,1636787,1636935,1.52251,ynfO
3,1696006,1696162,1.60706,uidA
4,1697057,1697307,1.81585,hdhA
5,1699674,1699857,1.75944,malX
6,1776350,1776521,2.04177,ydiF
7,2104847,2105014,1.89840,wbbJ
8,2107533,2107707,1.47040,glf
9,2990188,2990432,2.62642,yqeK


Функции генов

#uidA - Beta-glucuronidase

https://www.uniprot.org/uniprotkb/P05804/entry 

#hdhA - 7alpha-hydroxysteroid dehydrogenase

7alpha-hydroxysteroid dehydrogenase involved in the metabolism of bile acids. Catalyzes the NAD+-dependent oxidation of the 7alpha-hydroxy group of 7alpha-hydroxysteroids, such as the major human bile acids cholate and chenodeoxycholate, to the corresponding 7-oxosteroids. To a lesser extent, can also act on taurochenodeoxycholate, taurocholate and glycocholate


https://www.uniprot.org/uniprotkb/P0AET8/entry 

#malX - Maltooligosaccharide ABC transporter solute-binding lipoprotein

Part of an ABC transporter complex involved in the uptake of maltodextrins. Binds glycogen-derived linear maltooligosaccharides increasing in size from maltotriose to maltooctaose with the highest affinity for maltotriose. Has a very weak affinity for maltose. Has also a very low affinity for maltotetraitol, indicating that the binding is selective for maltooligosaccharides with an intact reducing end.

https://www.uniprot.org/uniprotkb/P19642/entry 

#ydiF - Acetate CoA-transferase

CoA transferase having broad substrate specificity for short-chain acyl-CoA thioesters with the activity decreasing when the length of the carboxylic acid chain exceeds four carbons. Exhibits high activity with acetoacetyl-CoA, propionyl-CoA, crotonoyl-CoA or butyryl-CoA as donors, with acetate as an acceptor. When acetyl-CoA is used as the donor, propionate, acetoacetate, butyrate, isobutyrate, and 4-hydroxybutyrate can be utilized as acceptors but not isovalerate. May play a role in short-chain fatty acid metabolism in E.coli.


https://www.uniprot.org/uniprotkb/Q8X5X6/entry 

#wbbJ - Putative lipopolysaccharide biosynthesis O-acetyl transferase

Putative O-acetyltransferase that transfers an O-acetyl group to the O antigen (lipopolysaccharide biosynthesis).

https://www.uniprot.org/uniprotkb/P37750/entry 

#glf - UDP-galactopyranose mutase

Catalyzes the interconversion through a 2-keto intermediate of uridine diphosphogalactopyranose (UDP-GalP) into uridine diphosphogalactofuranose (UDP-GalF).

https://www.uniprot.org/uniprotkb/P37747/entry 

#waaY - Lipopolysaccharide core heptose(II) kinase

Catalyzes the phosphorylation of heptose(II) of the outer membrane lipopolysaccharide core.

https://www.uniprot.org/uniprotkb/P27240/entry 



https://doi.org/10.1099/mic.0.000297

В одном исследовании было показано, что белок UxuR является репрессором синтеза ферментов, которые нужны для метаболизма гексуроновой кислоты. Также есть упоминание о том, что ранее была подтверждена важность этого белка для быстрого роста с d-глюкуронатом в качестве основного источника углерода и энергии.

https://www.uniprot.org/uniprotkb/P39161/entry

Также было найдено, что этот белок является регулятором транскрипции оперона Uxu, а именно репрессором uxuRBA оперона. Он относится к sugar utilization operons. Такие опероны кодируют белки, необходимые для утилизации соответствующих сахаров. Еще их называют деградационными оперонами. Они выключены в отсутствии этих сахаров. Механизмы метаболизма сахаров регулируются разными транскрипционными факторами. А именно эти факторы регулируют каталитические ферменты и траспортеры, задействованные в механизмы превращения и расщепления сахаров, то есть их метаболизма.

Utilization operon uxuRBA связан с генами утилизации галактуроната.

Таким образом, данный белок вероятно является транскрипционным фактором, который связывается с промоторами генов, ответственных за метаболизм сахаров(utilization genes), которые организованы в оперон uxuRBA, и регулирует их экспрессию. 

Функциями найденной группы генов являются кодирование каталитических ферментов и/или транспортеров метаболизма сахаров. Вполне вероятно, что UxuR контролирует уровень экспрессии этих (каких-то из них) генов.

Используя ChIPMunk, найдем мотив для выбранного белка.


Вытащим последовательности из генома по файлу с координатами пиков и передадим их в файл .fasta

In [ ]:
%cd /content/

/content


In [ ]:
!bedtools getfasta -fi Ecoli_K12_MG1655.fasta -bed /content/macs2/UxuR.bed -fo UxuR.mfa

In [ ]:
!head UxuR.mfa

>U00096.3:313256-313585
TTTAAAAAAGAGTGCACTTCGTAAGTGCCCTTATATAAATAACGAGTTTGGTCAACCAATTTTTTGACATGTATCACAAATTTGAATAGATGTATTACATCAACTATCTTTTATTGCACCAACGTCATTGATATATGTCGCCTGAAGTCAGTTCCGGGAATGAGTCTGATCTCAAGACTGGCCCAGTCCGGGCGTTGATTGGTGCTGAGGAGCATATCGCATCTCATCATAATGTCGTATCTCCTGGGGTGTTATACAAGATATCGTTGTTGGTGACCTGGGAGAGGAATTGAGTTCTATTAAACCGTCAACTATGCCGGATACATACT
>U00096.3:1215793-1215944
TCAAAATTCAGTGACTTTAGATTCTGCGGGAGCAATCACTCGTTACTTTGCAAAAGCTAACTTGCATACTCAGCAGGAAACTCTCGGGGAAATTGTGACTGAGATTTTGAAAGATGGACGTAATCTGAGTCGAAAGTCGCTTTGTGCCAAA
>U00096.3:1633815-1634134
ATTGATTGTGACAAAACTGGATCGCCTTGGTTGTAATGCAATGGATATCAGGAAAACAGTGGAACAACTGACCGAAACAGGTATCAGAGTGCATTGCTTAGCATTGGGGGGCATTGACCTGACCAGTCCAACAGGAAAAATGATGATGCAAGTAATTTCAGCAGTCGCTGAATTTGAACGAGACCTTTTACTTGAACGCACTCATTCCGGGATAGTAAGAGCCCGCGGCGCAGGGAAACGTTTTGGTCGACCACCTGTGTTAAATGAAGAACAGAAACAGGCGGTATTCGAACGAATTAAGTCAGGTGTAAGTATAAGT
>U00096.3:1636787-1636935
AACCCGCAACATCCGCTTTCCTAACCAAATGATTGAACAAATTAACATCGCTCTTGAGCAAAAAGGGTCCGGGAATTTCTCAGCCTGGGTCATTGA

Установим ChipMunk

In [ ]:
!wget https://autosome.org/ChIPMunk/chipmunk.jar

In [ ]:
!java -jar chipmunk.jar 7 10 yes oops s:UxuR.mfa

ru.autosome.ChIPMunk usage: <start_length> <stop_length> <verbose=(n)o|(y)es> <mode=(o)ops|zoops_factor=1.0|0.0> <ru.autosome.ChIPAct parameters>
PROG|ru.autosome.ChIPMunk V8 02022020
trying motif length: 7
ru.autosome.ChIPAct engine usage: <length> <s:simple_set> <o:ordered_set> <w:weighted_set> <p:peak_set> <m:summit_set> <r:rna_set> [<try_limit>=100] [<step_limit>=10>] [<iter_limit>=1] [<thread_count>=1] [<seeds_set>=random|filename.mfa] [<gc%>=0.5|auto] [<motif_shape>=flat|single|double] [<disable_log_weighting>]
using logarithmic sets weighting
using 15 sequences
gathering seeds...
gathered 100 seeds
PROG|ru.autosome.ChIPAct V8 02022020
found new GMLA with higher KDIC 0.7867034496915535 > 0.0
found new GMLA with higher KDIC 0.8091435085485743 > 0.7867034496915535
found new GMLA with higher KDIC 0.8096174879464358 > 0.8091435085485743
overall progress: 1% complete (motif discovery, length 7: 10.0% complete)
overall progress: 4% complete (motif discovery, length 7: 20.0% complete)
o

Таким образом, лучшим мотивом является TTGAGCAAAA с score = 9.931

WORD|3	53	TTGAGCAAAA	9.931206727341582	direct	0.9999999999999999

